In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,3911
2,Huelva,Confirmados PDIA 14 días,1733
3,Huelva,Tasa PDIA 14 días,"339,3095940619842"
4,Huelva,Confirmados PDIA 7 días,964
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,28
629,Municipio de Huelva sin especificar,Total Confirmados,68
630,Municipio de Huelva sin especificar,Curados,14


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  3911.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1029.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 570 personas en los últimos 7 días 

Un positivo PCR cada 318 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,3911.0,964.0,1733.0,510743.0,188.744633,339.309594,219.0
Huelva-Costa,1973.0,455.0,849.0,288115.0,157.923052,294.674002,116.0
Condado-Campiña,1256.0,343.0,515.0,155057.0,221.208975,332.135924,82.0
Huelva (capital),1029.0,252.0,452.0,143663.0,175.410509,314.625199,53.0
Lepe,299.0,72.0,136.0,27431.0,262.476760,495.789435,26.0
Almonte,210.0,92.0,133.0,24191.0,380.306726,549.791245,25.0
Gibraleón,124.0,56.0,79.0,12607.0,444.197668,626.635996,16.0
Sierra de Huelva-Andévalo Central,617.0,138.0,339.0,67571.0,204.229625,501.694514,14.0
Cartaya,96.0,24.0,58.0,19974.0,120.156203,290.377491,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),34.0,2.0,16.0,516.0,387.596899,3100.775194,0.0
Cortegana,112.0,14.0,89.0,4666.0,300.042863,1907.415345,1.0
Escacena del Campo,50.0,26.0,30.0,2284.0,1138.353765,1313.485114,4.0
Villarrasa,54.0,20.0,25.0,2176.0,919.117647,1148.897059,1.0
Almonaster la Real,20.0,3.0,18.0,1819.0,164.925783,989.554700,0.0
Calañas,23.0,14.0,20.0,2791.0,501.612325,716.589036,0.0
Palma del Condado (La),182.0,43.0,73.0,10761.0,399.591116,678.375616,9.0
Gibraleón,124.0,56.0,79.0,12607.0,444.197668,626.635996,16.0
Niebla,35.0,14.0,25.0,4117.0,340.053437,607.238280,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cerro de Andévalo (El),25.0,1.0,14.0,2364.0,42.301184,592.216582,0.0,0.071429
Granado (El),34.0,2.0,16.0,516.0,387.596899,3100.775194,0.0,0.125000
Cortegana,112.0,14.0,89.0,4666.0,300.042863,1907.415345,1.0,0.157303
Almonaster la Real,20.0,3.0,18.0,1819.0,164.925783,989.554700,0.0,0.166667
Villanueva de los Castillejos,30.0,2.0,10.0,2820.0,70.921986,354.609929,0.0,0.200000
Jabugo,19.0,1.0,5.0,2250.0,44.444444,222.222222,0.0,0.200000
Rosal de la Frontera,10.0,2.0,8.0,1697.0,117.855038,471.420153,1.0,0.250000
Lucena del Puerto,18.0,1.0,4.0,3371.0,29.664788,118.659152,0.0,0.250000
Bonares,11.0,1.0,4.0,6058.0,16.507098,66.028392,0.0,0.250000
